# Truck Routing

This script models truck-based delivery routing over eligible farm road segments using Pyomo and NetworkX.

## Summary

- Uses:
  - `Centroids.csv` (with `wt_1000` demand weights)
  - `Grid_Lines_Melted.csv` (with line connectivity and eligibility)
- Only lines with `Truck_Path == 1` are considered.
- Objective: minimize total truck energy (travel + optional idling) to serve all nonzero-demand centroids.

## Notes

- This run handles **one of six farm weight distributions**.
- To explore others, replace `wt_1000` with:
  - `wt_2000`, `wt_3000`, `wt_4000`, `wt_5000`, or `wt_6000`
- All demand-dependent logic (e.g., centroid filtering and coverage constraints) updates accordingly.

## Approach

- Lines are modeled as nodes in a graph; intersections are edges.
- Centroids are "covered" if at least one adjacent line is visited.
- A depot line (`9999`) must be included and connected.

## Output

- Total truck energy use in megajoules (MJ)
- List of visited lines
- Count of centroids served (should match demand list)

This approach simplifies routing logic while ensuring coverage, helping evaluate energy needs for each weight scenario efficiently.


In [7]:
# Wt_1000

import pandas as pd
import networkx as nx
from pyomo.environ import *

# --- Load Data ---
lines_df = pd.read_csv("Grid_Lines_Melted.csv")
centroids_df = pd.read_csv("Centroids.csv")

# Filter only truck-eligible lines
lines_df = lines_df[lines_df['Truck_Path'] == 1].copy()

# Only consider centroids with nonzero demand
demand_centroids = centroids_df[centroids_df['wt_1000'] > 0].copy()

# --- Collapse the melted dataset ---
# Each line_id now aggregates all centroids it serves and all neighbors (int_line_i)
line_groups = (
    lines_df.groupby('line_id')
    .agg({
        'length': 'first',  # Each line_id has a fixed length
        'cid': lambda x: list(set(x)),  # Unique centroids served
        'int_line_i': lambda x: list(set(x))  # Unique connected lines
    })
    .reset_index()
)

# Identify which lines actually serve demand centroids
centroid_to_lines = {}
for _, row in line_groups.iterrows():
    for cid in row['cid']:
        if cid in set(demand_centroids['cid']):
            centroid_to_lines.setdefault(cid, []).append(row['line_id'])

# Build undirected graph (lines as nodes, intersections as edges)
G = nx.Graph()
for _, row in line_groups.iterrows():
    for neighbor in row['int_line_i']:
        G.add_edge(row['line_id'], neighbor, length=row['length'])

# Depot line (start/end)
depot_line = 9999

# Energy cost (J/m) for the truck (F-150, 15 mph, 17 mpg)
c_truck = 4400  # From derivation: 7.1 MJ/mile -> ~4,400 J/m

# Idling energy parameters
idle_power = 23300  # J/s (from 0.7 gal/hr fuel consumption)
stop_energy = 0

# --- Pyomo Model ---
model = ConcreteModel()

# Sets
lines = set(G.nodes)
edges = list(G.edges)
model.L = Set(initialize=lines)         # Lines
model.E = Set(initialize=edges, dimen=2)  # Edges between lines

# Decision variables
model.x = Var(model.E, domain=NonNegativeIntegers)  # Number of times edge (i,j) is traversed
model.y = Var(model.L, domain=Binary)               # 1 if line is visited

# Objective: Minimize total energy (travel + idling)
def total_energy(model):
    travel_energy = sum(c_truck * G[i][j]['length'] * model.x[i, j] for (i, j) in model.E)
    return travel_energy
model.obj = Objective(rule=total_energy, sense=minimize)

# --- Constraints ---

# 1. Each centroid must be served by at least one eligible line
model.cover = ConstraintList()
for cid, lines_serving in centroid_to_lines.items():
    model.cover.add(sum(model.y[l] for l in lines_serving) >= 1)

# 2. Depot line must be visited
model.depot_visit = Constraint(expr=model.y[depot_line] == 1)

# 3. Flow balance for all lines (ensure lines used are connected)
model.flow = ConstraintList()
for l in lines:
    model.flow.add(sum(model.x[i, j] for (i, j) in model.E if i == l or j == l) >= model.y[l])

# 4. Depot entry/exit: ensure at least one connection at depot
model.depot_entry_exit = Constraint(
    expr=sum(model.x[i, j] for (i, j) in model.E if depot_line in (i, j)) >= 1
)

# --- Solve ---
solver = SolverFactory('cplex')  # or 'gurobi'
results = solver.solve(model, tee=True)

# --- Output ---
total_energy_val = value(model.obj)
print(f"Total truck energy (travel + stops): {total_energy_val/1e6:.2f} MJ")

# Lines visited
visited_lines = [l for l in lines if model.y[l].value > 0.5]
print(f"Visited lines (count={len(visited_lines)}): {visited_lines[:20]} ...")
print(f"Total visited lines: {len(visited_lines)}")

# Centroids served
covered_cids = set()
for cid, lines_serving in centroid_to_lines.items():
    if any(model.y[l].value > 0.5 for l in lines_serving):
        covered_cids.add(cid)
print(f"Centroids served: {len(covered_cids)} (should equal {len(demand_centroids)})")


(type: set).  This WILL potentially lead to nondeterministic behavior in Pyomo

Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\msela\AppData\Local\Temp\tmpphyq6_a8.cplex.log' open.
CPLEX> Problem 'C:\Users\msela\AppData\Local\Temp\tmpkwhu9lqv.pyomo.lp' read.
Read time = 0.00 sec. (0.18 ticks)
CPLEX> Problem name         : C:\Users\msela\AppData\Local\Temp\tmpkwhu9lqv.pyomo.lp
Objective sense      : Minimize
Variables            :    2899  [Binary: 841,  General Integer: 2058]
Objective nonzeros   :    2058
Linear constraints   :    1143  [Less: 841,  Greater: 301,  Equal: 1]
  Nonzeros           :    5617
  RHS nonzeros       :    